# Generate Raw DB

In [ ]:
import duckdb #create/update the duckdb database
import pandas as pd #import from excel
import os #iterate through files
import warnings #allows us to suppress an annoying irrelevant pandas warning
import re #also for the warning use case

# Print the current working directory for debugging
dir = os.getcwd()
print(dir)
# Debugging
# os.chdir('..')

# create a connection to the raw database
# If the db doesn't already exist, it'll be created at this location
con = duckdb.connect("dbs/raw_db.duckdb")

# prepare a sequence for identifying unique import rows
con.sql("create or replace sequence import_index start 1")

#Collect all D2D eBook files into one table
d2d_ebook = con.read_csv('raw/d2d*ebook*.csv', union_by_name = True, filename = True)
con.sql("create or replace table raw__d2d_ebook_sales as select *, nextval('import_index') as file_index from d2d_ebook")

#Collect all D2D Print files into one table
d2d_print = con.read_csv('raw/d2d*print*.csv', union_by_name=True, filename=True)
con.sql("create or replace table raw__d2d_print_sales as select *, nextval('import_index') as file_index from d2d_print")

#Parse the KDP tabs out of their .xlsx files
#First, fetch the matching files
dir = 'raw/' 
pattern = 'kdp'
matching_files = [f for f in os.listdir(dir) if pattern in f]

print("Matching KDP files found: ")
print(matching_files)

#Initialize a list for each table we'll be outputting
kenp_list = []
sales_list = []

#Loop through the matching files and tabs of interest
#Each loop adds entries to each table's list. The contributions from all files will be
#concatenated together into one table in the next step.

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning, module=re.escape('openpyxl.styles.stylesheet'))
    for file_name in matching_files:

        file_path = os.path.join('raw/',file_name)

        kenp = pd.read_excel(io=file_path,sheet_name="KENP",engine="openpyxl")
        kenp['file_name'] = pd.Series([file_name for x in range(kenp.size)])
        kenp['file_index'] = range(1, len(kenp) +1)

        kenp_list.append(kenp)

        sales = pd.read_excel(io=file_path,sheet_name="eBook Royalty",engine="openpyxl")
        sales['file_name'] = pd.Series([file_name for x in range(sales.size)])
        sales['file_index'] = range(1, len(sales) +1)
        sales_list.append(sales)

#Concat each list into one object
kenp_df  = pd.concat(kenp_list)
sales_df = pd.concat(sales_list)

#Create the tables
con.sql("create or replace table raw__kdp_kenp as select * from kenp_df")
con.sql("create or replace table raw__kdp_sales as select * from sales_df")

#Optionally, validate outputs
# con.table("raw__d2d_ebook_sales").show()
# con.table("raw__d2d_print_sales").show()
# con.table("raw__kdp_kenp").show()
# con.table("raw__kdp_sales").show()

# explicitly close the connection
con.close()

## View and validate outputs

In [ ]:
# # create a connection to the raw database
# con = duckdb.connect("dbs/raw_db.duckdb")

# con.table("raw__d2d_ebook_sales").show()
# con.table("raw__d2d_print_sales").show()
# con.table("raw__kdp_kenp").show()
# con.table("raw__kdp_sales").show()

# # explicitly close the connection
# con.close()

# Prepare Models Database

In [ ]:
from dbt.cli.main import dbtRunner, dbtRunnerResult

# Store the initial directory  
initial_dir = os.getcwd()

# change the working directory, down into the dbt folder
# print(os.getcwd())
os.chdir('author_db')


# initialize the dbt runner object
dbt = dbtRunner()

# create CLI args as a list of strings
seed_all = ["seed"]
run_all  = ["run"]
test_all = ["test"]

# run the command
print('*****DBT SEED*****')
res_seed: dbtRunnerResult = dbt.invoke(seed_all)
print('*****DBT RUN*****')
res_run:  dbtRunnerResult = dbt.invoke(run_all)
print('*****DBT TEST*****')
res_test: dbtRunnerResult = dbt.invoke(test_all)

# inspect the results
print('*****SEED RESULTS*****')
for r in res_seed.result:
    print(f"{r.node.name}: {r.status}")

print('*****RUN RESULTS*****')
for r in res_run.result:
    print(f"{r.node.name}: {r.status}")

print('*****TEST RESULTS*****')
for r in res_test.result:
    print(f"{r.node.name}: {r.status}")

# return to the initial working directory
os.chdir(initial_dir)
print(os.getcwd())
